# PSyclone tutorial: NEMO API Example 1 - Translation to PSyIR

This example introduces PSyclone's NEMO API and shows how Fortran code is translated into PSyclone's language independent internal representation (the PSyIR) and then translated back into code.

First let's specify a simple Fortran code in a Python string. PSyclone typically reads codes from files but it is easier to demonstrate what is happening using a string.

Notice that the code consists of nested loops containing accesses to multi-dimensional arrays. This code structure is typical of finite difference and finite volume codes running on regular meshes and is what the NEMO API expects. If code is written in a different way e.g. with indirection to support irregular meshes, then the NEMO API will not be useful and should not be used.

Also notice that the outer loops have indices `jk` with bounds from `1` to `jpk`, the middle loops have indices `jj` with bounds from `1` to `jpj` and the inner loops have indices `ji` with bounds from `1` to `jpi`. The NEMO coding rules enforce this naming convention throughout the code where `ji` corresponds to iterating over longitude, `jj` corresponds to iterating over latitude and `jk` corresponds to iterating over levels. PSyclone uses this naming convention to determine the type of loop. If a different naming scheme is used in a different code then PSyclone can be changed to use these instead. If you are interested you can find the mappings used in the PSyclone [config file](https://github.com/stfc/PSyclone/blob/master/config/psyclone.cfg). See the [user guide](https://psyclone.readthedocs.io/en/stable/configuration.html) for more details.

In [ ]:
code = '''program test
  implicit none
  integer, parameter :: jpi=10, jpj=10, jpk=10
  real, dimension(jpi,jpj,jpk) :: a,b
  integer :: ji,jj,jk
  do jk=1,jpk
    do jj=1,jpj
      do ji=1,jpi
        b(ji,jj,jk) = 0.0
      end do
    end do
  end do
  do jk=1,jpk
    do jj=1,jpj
      do ji=1,jpi
        a(ji,jj,jk) = b(ji,jj,jk)
      end do
    end do
  end do
end program test'''

In [ ]:
from fparser.common.readfortran import FortranStringReader
reader = FortranStringReader(code)
from fparser.two.parser import ParserFactory
parser = ParserFactory().create(std="f2003")
parse_tree = parser(reader)

Once the code has been parsed by the Fortran parser the next step is for PSyclone to take the code and transform it into PSyclone's internal representation, the PSyIR.

As discussed in the tutorial introduction, PSyclone was originally developed to support the concept of 3 separate layers, an Algorithm layer that calls a PSy layer which in turn calls a Kernel layer. The Algorithm and Kernel layers are written by scientists and the PSy layer is generated by PSyclone and contains any parallel code. The Algorithm layer specifies which kernels should be called in regions of code called invoke's. For each invoke PSyclone creates PSy layer code that is called by the algorithm layer and in turn the PSy layer code calls the required kernels.

When PSyclone is run it creates a top level `PSy` object which conceptually represents the PSy layer. Within the `PSy` object there are one or more `invoke` objects. Each `invoke` object contains a PSyIR `schedule` which specifies how the PSy layer code will call the required kernels.

In the NEMO API we are dealing with existing code so there is no Algorithm, PSy or Kernel layer and there are no invoke regions specified by scientists. The approach taken is to treat the input code as being from a single invoke region (so there is no concept of an Algorithm layer) with the input code representing a pre-existing PSy and Kernel layer mixed together. These layers are then logically separated by specifying that code within loop nests defines kernels in the PSyIR.

OK, so let's create the PSyIR and associated objects. The objects are contained within the `psy` object hierarchy.

In [ ]:
from psyclone.psyGen import PSyFactory
psy = PSyFactory("nemo").create(parse_tree)

As mentioned earlier there is only one invoke region in the NEMO API. The code below checks this (as the invoke objects is kept as a list in `invoke_list`).

In [ ]:
print(len(psy.invokes.invoke_list))

Now let's get access to the PSyIR schedule from the single invoke object.

In [ ]:
invoke = psy.invokes.invoke_list[0]
schedule = invoke.schedule

The PSyIR is a language-independent representation of the PSy and Kernel layers. It consists of a hierarchy of nodes. As such it is similar to the fparser2 node hierarchy, discussed in the fparser2 section of this tutorial, but fparser2 produces a tree that is specific to Fortran. For more details of working with the PSyIR, please see the PSyIR [tutorial](../psyir/psyir_example1.ipynb).

The `view()` method provides a textual view of the PSyIR tree:

In [ ]:
print(schedule.view())

If you compare the view of the PSyIR schedule with the original code, the relationship should be relatively clear. As mentioned earlier, notice that PSyclone has determined the type of the loops (see `Loop[type= ...]`).

Also notice how the bodies of the various loop nests are now specified as being (inlined) Kernels.

So what has this bought us? We started with Fortran code, we parsed that into a Fortran parse tree and have now translated that into PSyIR.

The reasons for doing this in the NEMO API are:

* we now have some domain knowledge about the loops (whether they iterate over latitude, longitude or levels),

* the other APIs also use the PSyIR and this allows us to share transformations/optimisations between the different APIs and

* it is possible to output code in different languages - for example, it is not possible to run Fortran code on FPGA's. 

(Note however that for the NEMO API, the output is currently limited to Fortran.)

In later examples we will look at transforming/optimising the code but for the moment we simply output the unmodified code in Fortran:

In [ ]:
print(str(psy.gen))

Let's move on to the [next example](nemo_example2.ipynb)...